In [3]:
import json
from portfolio import Portfolio
from optimizer import PortfolioOptimizer
from simulations import MonteCarloSimulation
from utils import (
    get_simulation_insights,
    display_optimal_weights
)

#Snowpark lib
from snowflake.snowpark import Session

from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

config = {
    "TICKERS" : [], "START_DATE" : '2020-01-01', "END_DATE" : '2023-01-01', "INITIAL_INVESTMENT" : 1000,
    "NUM_SIMULATIONS" : 10000, "TIME_HORIZON" : 252, "RISK_FREE_RATE" : 0.02, "WEIGHTS" : None, "OPTIMIZE" : True, "BALANCED" : False
}

In [4]:
table_name = 'STOCKS_DATA'
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [7]:
tickers = ["MTFS", "ALPA", "OOGGL", "TCA", "ABC"]
config["TICKERS"] = tickers

In [8]:
config

{'TICKERS': ['MTFS', 'ALPA', 'OOGGL', 'TCA', 'ABC'],
 'START_DATE': '2020-01-01',
 'END_DATE': '2023-01-01',
 'INITIAL_INVESTMENT': 1000,
 'NUM_SIMULATIONS': 10000,
 'TIME_HORIZON': 252,
 'RISK_FREE_RATE': 0.02,
 'WEIGHTS': None,
 'OPTIMIZE': True,
 'BALANCED': False}

In [13]:
def load_data(df, tickers: list):
    for ticker in tickers:
        data = df[df["STOCK"].apply(lambda x: x in tickers)]
    return data

In [ ]:

def main(config=config):
    
    # Extract configuration parameters
    tickers = config.get('tickers', [])
    start_date = config.get('start_date')
    end_date = config.get('end_date')
    initial_investment = config.get('initial_investment', 1000)
    num_simulations = config.get('num_simulations', 10000)
    time_horizon = config.get('time_horizon', 252)
    risk_free_rate = config.get('risk_free_rate', 0.0)
    custom_weights = config.get('weights')
    optimization_config = config.get('optimization', {})
    optimize = optimization_config.get('optimize', False)
    balanced = optimization_config.get('balanced', True)
    
    # Load data
    stock_data = load_data(df, tickers)
    
    # Create portfolio
    portfolio = Portfolio(stock_data)
    portfolio.calculate_returns()
    
    # Annualize returns and covariance
    expected_returns = portfolio.returns.mean() * 252
    covariance_matrix = portfolio.returns.cov() * 252
    
    # Determine weights
    if optimize:
        optimizer = PortfolioOptimizer(
            expected_returns,
            covariance_matrix,
            risk_free_rate=risk_free_rate
        )
        if balanced:
            optimal_weights = optimizer.minimize_volatility(target_return=expected_returns.mean())
            print("\nOptimal Balanced Portfolio Weights:")
        else:
            optimal_weights = optimizer.maximize_sharpe_ratio()
            print("\nOptimal Portfolio Weights to Maximize Sharpe Ratio:")
        display_optimal_weights(stock_data.columns, optimal_weights)
        weights = optimal_weights
    elif custom_weights:
        weights = custom_weights
        print("\nUsing Custom Weights:")
        display_optimal_weights(stock_data.columns, weights)
    else:
        num_assets = len(expected_returns)
        weights = [1.0 / num_assets] * num_assets
        print("\nUsing Equal Weights:")
        display_optimal_weights(stock_data.columns, weights)
    
    # Perform Monte Carlo Simulation
    simulation = MonteCarloSimulation(portfolio.returns, initial_investment, weights)
    all_cumulative_returns, final_portfolio_values = simulation.run_simulation(
        num_simulations, time_horizon
    )
    
    # Analyze Results
    print_simulation_insights(final_portfolio_values, initial_investment)
    
    # Plot results
    plot_simulation_results(all_cumulative_returns, final_portfolio_values)